<a href="https://colab.research.google.com/github/neoyipeng2018/acct725_teachingmaterials/blob/main/Lesson4A_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 8 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-20 03:25:07 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-20 03:25:07 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.6.2: Fast Qwen2 patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-1.5b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chun

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.6.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<think>
...
</think>
<answer>
...
</answer>
"""

In [ ]:
from datasets import Dataset, DatasetDict

# 1️⃣  Put your 30 rows in a list of dicts.
#     For clarity I show two rows; extend to all 30.
data = [
    {
        "text": "Advanced economy, late-cycle boom. After a decade of low duties, Norland’s centrist coalition slaps a sudden 5 ppt across-the-board surcharge on all imported intermediate inputs to “re-shore supply chains.” Domestic demand is already running above potential, wages are sticky and the currency jumps on announcement.",
        "label": 2
    },
    {
        "text": "Mid-income exporter in mild recession. Paragonia lifts consumer-goods tariffs by 4 ppt to shield household-appliance plants while unemployment sits at 9 %. Monetary policy stays accommodative.",
        "label": 1
    },
    {
        "text": "Small open economy, tariff cut during slump. Baltica removes its 2 ppt duty on electronics components, hoping to revive contract-manufacturing jobs.",
        "label": 0
    },
    {
        "text": "G-7 boom plus retaliation. The U.S. raises auto-part (input) tariffs 10 ppt; Canada, Mexico and the EU respond in kind within weeks. Auto firms face cost spikes and shrinking export markets simultaneously.",
        "label": 2
    },
    {
        "text": "EMDE, commodity bust. Copper-dependent Andor slaps a 6 ppt tariff on imported finished furniture to spur local sawmills during a deep downturn in global metal prices.",
        "label": 1
    },
    {
        "text": "Advanced, targeted output tariff. Sweden adds a 3 ppt levy on luxury handbags only. Inputs remain duty-free and the move happens during flat growth.",
        "label": 0
    },
    {
        "text": "Boom-time input tariff with currency peg. The UAE imposes 5 ppt on construction steel; the dirham’s dollar peg blocks offsetting depreciation, so costs pass straight to domestic prices.",
        "label": 2
    },
    {
        "text": "Emerging Asia, expansion, small hike. Vietnam nudges seafood tariffs up 1 ppt to defend coastal jobs; foreign share of value-added is low, and inputs are exempt.",
        "label": 0
    },
    {
        "text": "Advanced cyclical downturn, big output cut. Italy, in recession, cuts auto tariffs by 4 ppt as part of an EU stimulus plan, accepting temporary job churn.",
        "label": 0
    },
    {
        "text": "Resource-rich EM, election year. Brazil raises farm-machinery input tariffs 7 ppt during a soy-price boom to please rural lobbies; central bank hikes rates to curb the inflationary pass-through.",
        "label": 2
    },
    {
        "text": "Island micro-state, tourism shock. Fiji lifts non-essential consumer-goods tariffs 3 ppt after a cyclone crushes hotel occupancy; IMF support offsets FX stress.",
        "label": 1
    },
    {
        "text": "Advanced stagflation scare. The UK introduces a 4 ppt carbon border tax on steel (input) during a supply-side inflation spike; BOE tightens aggressively.",
        "label": 2
    },
    {
        "text": "EMDE, WTO-driven liberalisation. Kenya phases out its 2 ppt textile duty while growth is solid but credit tight; firms redeploy labour from protected mills to logistics.",
        "label": 0
    },
    {
        "text": "Advanced, recession + input tariff. Japan raises semiconductor wafers 2 ppt amid a tech slump to rescue domestic fabs; slack labour markets mute wage pressure.",
        "label": 1
    },
    {
        "text": "Frontier market boom, broad hike. Laos adds 5 ppt across most HS lines (inputs & outputs) as FDI surges; kip appreciates, eroding hoped-for trade gains.",
        "label": 2
    },
    {
        "text": "Advanced, services focus. France taxes only digital-ad services (output) 3 ppt; physical-goods supply chains unchanged.",
        "label": 0
    },
    {
        "text": "EM, high import content. Turkey adds 6 ppt on petrochemical feedstocks (inputs) while GDP grows 4 %; lira already weak, so prices in TRY soar.",
        "label": 2
    },
    {
        "text": "Advanced, counter-cyclical cut. Australia scraps its 5 ppt wine tariff in the middle of a drought-induced recession, inviting cheaper imports but boosting real incomes.",
        "label": 0
    },
    {
        "text": "Small EM, modest consumer tariff. Jordan hikes smartphone duties 2 ppt to fund refugee relief; substitution toward second-hand phones limits macro fallout.",
        "label": 0
    },
    {
        "text": "Advanced, sector-specific input hike. Germany adds 4 ppt on imported robotics servos during peak manufacturing output; export competitiveness erodes.",
        "label": 2
    },
    {
        "text": "EM, election-cycle give-away. Mexico cuts auto parts tariffs 3 ppt six months before polls in a flat economy, signalling openness.",
        "label": 0
    },
    {
        "text": "Advanced, mild boom, output tariff. Canada levies 2 ppt on finished furniture; domestic sawmills already at capacity and input chain untouched.",
        "label": 1
    },
    {
        "text": "Fragile EM, IMF-backed reform. Egypt replaces a blanket 6 ppt duty with a VAT, lowering average tariffs 3 ppt during recovery.",
        "label": 0
    },
    {
        "text": "Advanced, large mixed basket. South Korea imposes 6 ppt on both intermediate chips and finished phones at full employment; won appreciates.",
        "label": 2
    },
    {
        "text": "EM, commodity boom, limited scope. Nigeria raises 3 ppt tariff on imported chocolates only; oil revenues crowd out broader effects.",
        "label": 0
    },
    {
        "text": "Advanced, defence shock. The U.S. slaps 25 ppt steel/aluminium duties in a late-cycle surge, spilling into autos and machinery inputs.",
        "label": 2
    },
    {
        "text": "EM, recession, small input hike. Argentina adds 2 ppt on chemical feedstocks amid a currency crisis; output already 3 % below trend.",
        "label": 1
    },
    {
        "text": "Advanced, post-crisis liberalisation. Spain eliminates 2 ppt solar-panel duties during early recovery, accelerating green-energy capex.",
        "label": 0
    },
    {
        "text": "EM, boom, agriculture inputs. Thailand raises fertiliser tariffs 4 ppt in peak planting season; rice exporters face cost squeeze.",
        "label": 1
    },
    {
        "text": "Advanced, synchronized retaliation spiral. The EU, U.S., and China each add 5 ppt on aerospace components (inputs) in a hot global cycle, triggering cascading supply-chain delays.",
        "label": 2
    }
]

# 2️⃣  Map the string labels to integers (0/1/2 is conventional)
label2id = {0: "Low", 1: "Moderate", 2: "High"}
for row in data:
    row["label"] = label2id[row["label"]]

# 3️⃣  Slice into train / validation splits
train_rows = data[:20]      # first 20
valid_rows = data[20:]      # last 10

# 4️⃣  Build the DatasetDict
ds = DatasetDict({
    "train":  Dataset.from_list(train_rows),
    "validation": Dataset.from_list(valid_rows)
})

# 5️⃣  (Optional) Inspect -- should show 20 / 10
print(ds)
print(ds["train"][0])

# 6️⃣  (Optional) Push to the Hub under your namespace.
# ds.push_to_hub("your-username/tariff-impact-mini")

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 10
    })
})
{'text': 'Advanced economy, late-cycle boom. After a decade of low duties, Norland’s centrist coalition slaps a sudden 5 ppt across-the-board surcharge on all imported intermediate inputs to “re-shore supply chains.” Domestic demand is already running above potential, wages are sticky and the currency jumps on announcement.', 'label': 'High'}


In [ ]:
dataset = ds['train'].map(lambda x: {
    'prompt': [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': f"""What is the economic impact of the following situation? Output High, Moderate or Low. Reason step by step within <think></think> tags and Answer the question below within <answer></answer> tags. ##SITUATION: {x['text']}"""}
    ],
    'answer': str(x['label'])
},remove_columns=["text","label"])

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
dataset[0]

{'prompt': [{'content': '\nRespond in the following format:\n<think>\n...\n</think>\n<answer>\n...\n</answer>\n',
   'role': 'system'},
  {'content': 'What is the economic impact of the following situation? Output High, Moderate or Low. Reason step by step within <think></think> tags and Answer the question below within <answer></answer> tags. ##SITUATION: Advanced economy, late-cycle boom. After a decade of low duties, Norland’s centrist coalition slaps a sudden 5 ppt across-the-board surcharge on all imported intermediate inputs to “re-shore supply chains.” Domestic demand is already running above potential, wages are sticky and the currency jumps on announcement.',
   'role': 'user'}],
 'answer': 'High'}

In [ ]:
import re
import math
from datasets import load_dataset, Dataset
from random import random

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_xml_reasoning(text: str) -> str:
    reasoning = text.split("<think>")[-1]
    reasoning = reasoning.split("</think>")[0]
    return reasoning.strip()

# Your modified reward function
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]

    extracted_responses_final = []
    for r in extracted_responses:
        matches = re.findall(r'\b(Moderate|High|Low)\b', r)
        extracted_responses_final.append(matches[-1] if matches else r)

    rewards = []
    for r_text, r_extracted, a in zip(responses, extracted_responses_final, answer):
        reward = 2.0 if str(r_extracted).lower() == str(a).lower() else 0.0
        rewards.append(reward)

    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")

    return rewards

def count_xml(text) -> float:
    count = 0.0
    if text.count("<think>\n") == 1:
        count += 0.125
    if text.count("\n</think>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

### Train

In [ ]:
import os
os.environ['WANDB_API_KEY'] = 'c6003ee33aa4304f3b55296330fbf7595523dcfc'

In [ ]:
import wandb
wandb.login()

wandb.init(
    project="acct725",
    name="reasoning",
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


profiling/Time taken: UnslothGRPOTrainer._prepare_inputs,█▁▁█▁▁▇▁▁▁▁▁▁▅▁▆▁▁▁▇▁▁▆▁▁▇▁▁▆▁▁▆▁▁▁▁▁▁▁▁
profiling/Time taken: UnslothGRPOTrainer.correctness_reward_func,▅▃▃▃▁▃▁▂█▂▃▁▂
profiling/Time taken: UnslothGRPOTrainer.vLLM.generate,█▇▅▄▁▄▅▃▅▄▄▅▅
profiling/Time taken: UnslothGRPOTrainer.xmlcount_reward_func,▆█▁▂▂▅▄▂▆▂▃▂▂
train/completion_length,▅▁▄▃▁▄▄▁█▂▄▃▃
train/completions/clipped_ratio,▆▃▆▃▁▁▆▁█▃▃▆▃
train/completions/max_length,████▁▆█▄█████
train/completions/max_terminated_length,▁▇█▄▂▇▃▅▇▇▅▂▇
train/completions/mean_length,▅▁▄▃▁▄▄▁█▂▄▃▃
train/completions/mean_terminated_length,▅▁▄▃▃▆▃▃█▂▅▂▃
train/completions/min_length,█▄▆▁▅▅▅▃▅▂▅▆▂


In [ ]:
from trl import GRPOConfig, GRPOTrainer

max_prompt_length = 512

training_args = GRPOConfig(
    use_vllm = False, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory, originally 6
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 5, # Set to 1 for a full training run
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20 | Num Epochs = 5 | Total steps = 25
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 9,232,384/1,500,000,000 (0.62% trained)


-------------------- Question:
What is the economic impact of the following situation? Output High, Moderate or Low. Reason step by step within <think></think> tags and Answer the question below within <answer></answer> tags. ##SITUATION: Advanced, sector-specific input hike. Germany adds 4 ppt on imported robotics servos during peak manufacturing output; export competitiveness erodes. 
Answer:
High 
Response:
In <think> analysis, let's break it down as follows:</think>

  1. High economic impact:
    <answer>
Concerns about unanticipated consequences:
    - Unmentioned economic forecasts are considered high-strike situations.
    - Stakeholders are skeptical of the validity and timeliness of forecasts.
    - The economic lawyer criticizes that forecasts cannot be trusted because they are based on uncertainty and therefore unreliable statistics. Instead, he recommends baselining historical company performance.
    - Chapter 2: Ethical Question: Employees' performance is subject to exte

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / xmlcount_reward_func / mean,rewards / xmlcount_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
1,0.000000,0.937063,0.890788,292.687500,99.000000,512.000000,0.125000,261.357147,99.000000,403.000000,0.000020,-0.187937,0.458086,1.125000,1.024695
2,0.000000,0.303813,0.917935,248.812500,125.000000,512.000000,0.062500,231.266678,125.000000,431.000000,0.000021,-0.446187,0.757006,0.750000,1.000000
3,0.000000,0.243187,0.783886,280.000000,116.000000,512.000000,0.062500,264.533356,116.000000,480.000000,0.000022,-0.381813,0.607586,0.625000,0.957427
4,0.000000,0.436500,0.567771,270.687500,144.000000,512.000000,0.062500,254.600006,144.000000,465.000000,0.000020,-0.188500,0.606108,0.625000,0.957427
5,0.000000,0.590312,1.175351,240.437500,78.000000,512.000000,0.062500,222.333344,78.000000,403.000000,0.000025,-0.284688,0.383772,0.875000,1.024695
6,0.000000,0.097562,0.884583,231.937500,114.000000,353.000000,0.062500,223.866684,114.000000,353.000000,0.000022,-0.277438,0.384718,0.375000,0.806226
7,0.000000,1.072000,1.256748,224.250000,92.000000,512.000000,0.062500,205.066681,92.000000,436.000000,0.000023,-0.178000,0.404798,1.250000,1.000000
8,0.000000,0.231125,0.917662,315.500000,109.000000,512.000000,0.187500,270.153870,109.000000,507.000000,0.000018,-0.143875,0.374430,0.375000,0.806226
9,0.000000,0.182125,0.975917,280.687500,145.000000,512.000000,0.062500,265.266693,145.000000,445.000000,0.000023,-0.317875,0.561521,0.500000,0.894427
10,0.000000,0.741313,0.736541,286.312500,109.000000,512.000000,0.062500,271.266693,109.000000,507.000000,0.000021,-0.383687,0.572279,1.125000,1.024695


-------------------- Question:
What is the economic impact of the following situation? Output High, Moderate or Low. Reason step by step within <think></think> tags and Answer the question below within <answer></answer> tags. ##SITUATION: Advanced stagflation scare. The UK introduces a 4 ppt carbon border tax on steel (input) during a supply-side inflation spike; BOE tightens aggressively. 
Answer:
High 
Response:
<think>
In this scenario, the primary economic impact focuses on the interaction between tightening monetary policy and a carbon border tax. High inflation is countered by a BOE's aggressive tightening, which raises interest rates to combat inflation. Businesses and workers struggle against supply-side inflation, characterized by rising utility prices and increased costs.

Moderate supply-side inflation is offset by a pricing strategy that acts as a shock absorber to overall output. This collaboration maintains an insulated effect on prices within a relatively lower, yet not 

TrainOutput(global_step=25, training_loss=9.438395500183106e-07, metrics={'train_runtime': 759.2969, 'train_samples_per_second': 0.132, 'train_steps_per_second': 0.033, 'total_flos': 0.0, 'train_loss': 9.438395500183106e-07})

In [ ]:
model.save_lora("tariff_qwen")

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)

In [ ]:
p = """Advanced economy, retaliation cycle. In June 2025, Canada imposes new protective tariffs on steel and aluminum imports in direct response to the United States doubling its steel and aluminum tariffs from 25% to 50%. Canada maintains its baseline 25% duty but signals further increases may follow pending negotiations. The new tariffs are structured to target imports from non-FTA countries, especially China, and Ottawa announces that federal infrastructure and auto procurement will preferentially use Canadian steel and aluminum. The moves come as global trade tensions escalate and both economies operate near full employment."""
# p = """Major economies, complex agreement. In June 2025, the United States and China finalize a framework agreement to stabilize trade relations after a period of escalating tensions. The deal establishes a multi-layered tariff structure, imposing a new 10% baseline reciprocal tariff on all Chinese goods. This is stacked on top of existing measures, such as the 25% Section 301 tariffs and a 20% tariff on goods linked to fentanyl precursors, leading to cumulative rates as high as 35%. While aiming to de-escalate the trade war, the agreement introduces a new, complex system of managed trade between the world's two largest economies."""

In [ ]:
processed_batch = [tokenizer.apply_chat_template([
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""What is the economic impact of the following situation? Output High, Moderate or Low. Reason step by step within <think></think> tags and Answer the question below within <answer></answer> tags.

## SITUATION
{p}"""}], tokenize=False, add_generation_prompt=True)]


outputs = model.fast_generate(
            processed_batch,
            sampling_params=sampling_params,
        )

batch_results = [output.outputs[0].text for output in outputs]
batch_results[0]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"**Think:** The situation involves Canada's imposition of retaliatory tariffs in response to the United States' increased tariffs on steel and aluminum. The Canada-US trade relationship is likely to be impacted, and the economic impact depends on the extent of retaliation and the response of other countries. The escalation of global trade tensions and Canada's strategic decision to increase steel and aluminum prices for its own use indicate a significant economic impact.\n\n**Answer:** High. The escalation of trade tensions and the imposition of retaliatory tariffs suggest a high economic impact. The increased costs for imported steel and aluminum will likely lead to higher prices for goods and services in Canada, potentially slowing economic growth. The increased use of Canadian steel and aluminum in federal infrastructure and auto procurement will also have significant economic implications, particularly if it leads to a boost in the domestic steel and aluminum industries.\n\nOverall

Now load GRPO

In [ ]:
lora_request = model.load_lora("tariff_qwen")

processed_batch = [tokenizer.apply_chat_template([
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""What is the economic impact of the following situation? Output High, Moderate or Low. Reason step by step within <think></think> tags and Answer the question below within <answer></answer> tags.

## SITUATION
{p}"""}], tokenize=False, add_generation_prompt=True)]

outputs = model.fast_generate(
            processed_batch,
            sampling_params=sampling_params,
            lora_request=lora_request
        )


batch_results = [output.outputs[0].text for output in outputs]
batch_results[0]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"<think>\nIn this situation, Canada is responding to the increased tariffs imposed by the United States. The imposition of higher tariffs on steel and aluminum imports from non-FTA countries, particularly China, is a response to the US's retaliation. The announcement of preferential use of Canadian steel and aluminum in infrastructure and auto procurement further intensifies this response. This situation involves retaliatory measures in a trade war, which typically leads to higher costs for businesses and consumers.\n</think>\n\n<answer>\nModerate\n</answer>\n\nThe economic impact of this situation is moderate. High tariffs on steel and aluminum from non-FTA countries can lead to higher costs for businesses and consumers in Canada, particularly in sectors that rely heavily on imported steel and aluminum. This can lead to higher prices for goods and services, which can have a negative impact on economic growth and employment. However, the preferential use of Canadian steel and aluminum 

In [ ]:
p = """Major economies, complex agreement. In June 2025, the United States and China finalize a framework agreement to stabilize trade relations after a period of escalating tensions. The deal establishes a multi-layered tariff structure, imposing a new 10% baseline reciprocal tariff on all Chinese goods. This is stacked on top of existing measures, such as the 25% Section 301 tariffs and a 20% tariff on goods linked to fentanyl precursors, leading to cumulative rates as high as 35%. While aiming to de-escalate the trade war, the agreement introduces a new, complex system of managed trade between the world's two largest economies."""

In [ ]:
processed_batch = [tokenizer.apply_chat_template([
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""What is the economic impact of the following situation? Output High, Moderate or Low. Reason step by step within <think></think> tags and Answer the question below within <answer></answer> tags.

## SITUATION
{p}"""}], tokenize=False, add_generation_prompt=True)]


outputs = model.fast_generate(
            processed_batch,
            sampling_params=sampling_params,
        )

batch_results = [output.outputs[0].text for output in outputs]
batch_results[0]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"Think: The situation involves a major trade agreement between two major economies, the United States and China, which has the potential to significantly impact global trade and economic relations. We need to analyze the details of the agreement to determine the economic impact.\n\n1. **Baseline Tariff**: A 10% baseline reciprocal tariff on all Chinese goods.\n   - This is a straightforward tariff and would have a moderate impact on China's economy, affecting imports and potentially leading to retaliatory measures from China.\n\n2. **Existing Measures**: The Section 301 tariffs and the 20% tariff on goods linked to fentanyl precursors.\n   - These measures are already in place and could be augmented by the 10% baseline tariff, but their cumulative effect is more complex. They would likely have a significant economic impact on China, especially in industries reliant on imports.\n\n3. **Multi-layered Tariff Structure**: The complex system of managed trade.\n   - The multilayered structur

In [ ]:
lora_request = model.load_lora("tariff_qwen")

processed_batch = [tokenizer.apply_chat_template([
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""What is the economic impact of the following situation? Output High, Moderate or Low. Reason step by step within <think></think> tags and Answer the question below within <answer></answer> tags.

## SITUATION
{p}"""}], tokenize=False, add_generation_prompt=True)]

outputs = model.fast_generate(
            processed_batch,
            sampling_params=sampling_params,
            lora_request=lora_request
        )


batch_results = [output.outputs[0].text for output in outputs]
batch_results[0]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"<think>\nIn June 2025, the United States and China finalize a framework agreement to stabilize trade relations. The deal establishes a multi-layered tariff structure, imposing a new 10% baseline reciprocal tariff on all Chinese goods. This is stacked on top of existing measures, such as the 25% Section 301 tariffs and a 20% tariff on goods linked to fentanyl precursors, leading to cumulative rates as high as 35%. The agreement introduces a new, complex system of managed trade between the world's two largest economies.\n</think>\n\n<answer>\nThe economic impact of this situation is High. The complex agreement between the United States and China will lead to high tariffs on Chinese goods, resulting in a significant increase in production costs for the Chinese economy. This will have a negative impact on the Chinese economy, leading to a decrease in economic growth and a decrease in consumer spending. On the other hand, the United States will benefit from the increased revenue from the t